In [ ]:
%pip install tensorflow==2.12 opencv-python mediapipe sklearn matplotlib


In [ ]:
# § Test if your webcam's feed is accessible
import cv2

cap = cv2.VideoCapture(0)
# ! if this does not open a window with your webcam, try incrementing the '0' to find the right device number
# ? You can replace the '0' with a file path to open a video saved in your local files

while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow("OpenCV feed", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
# § Load labels as dict
import json
import os

data_path = r'downloads/data'
labels_file = r'labels.json'

with open(os.path.join(data_path, labels_file), "r") as labels_reader:
    labels = json.load(labels_reader)

labels


In [ ]:
# § Vectorise a landmark sequence
import numpy as np

data_path = r'downloads/data'
landmarks_folder = r'landmarks'


def vectorize_landmark(landmark: np.ndarray, *, include_face: bool = True, include_pose: bool = True,
                       include_righth: bool = True, include_lefth: bool = True) -> np.ndarray:
    return np.concatenate(landmark[[include_face, include_pose, include_lefth, include_righth]])


def vectorize_landmark_frames(landmark_frames: np.ndarray, *, include_face: bool = True, include_pose: bool = True,
                              include_righth: bool = True, include_lefth: bool = True) -> np.ndarray:
    for i, landmarks in enumerate(landmark_frames):
        landmark_frames[i] = vectorize_landmark(landmarks, include_face=include_face, include_pose=include_pose,
                                                include_lefth=include_lefth, include_righth=include_righth)
    return landmark_frames


lm = np.load(r'downloads/data/landmarks/00597.npy', allow_pickle=True)
vlm = vectorize_landmark_frames(lm, include_face=False)
vlm


In [ ]:
# return the true number of dim for a numpy jagged array
def dim_number_jaggedArray(arr):
    dimensions = 0
    arr_copy = arr.copy()

    while True:
        try:
            _ = arr_copy[0]
            dimensions += 1
            arr_copy = arr_copy[0]
        except (TypeError, IndexError):
            break

    return dimensions

# return which first dim of a numpy jagged array is jagged, -1 if no jagged dimension


def find_jagged_dimension(arr):
    if (arr.ndim != (dim_number_jaggedArray(arr))):
        return arr.ndim+1
    else:
        return -1


In [ ]:
# not very useful function
def get_jagged_structure(arr):
    structure = []
    if isinstance(arr, np.ndarray) and arr.ndim > 0:
        for subarr in arr:
            sub_structure = get_jagged_structure(subarr)
            structure.append(len(sub_structure))
    else:
        return []

    return structure
